In [26]:
import numpy as np
import inspect

source_code = inspect.getsource(np.sum)

print("Source code of np.array:")
print(np.sum.__doc__)
   


Source code of np.array:

    Sum of array elements over a given axis.

    Parameters
    ----------
    a : array_like
        Elements to sum.
    axis : None or int or tuple of ints, optional
        Axis or axes along which a sum is performed.  The default,
        axis=None, will sum all of the elements of the input array.  If
        axis is negative it counts from the last to the first axis.

        .. versionadded:: 1.7.0

        If axis is a tuple of ints, a sum is performed on all of the axes
        specified in the tuple instead of a single axis or all the axes as
        before.
    dtype : dtype, optional
        The type of the returned array and of the accumulator in which the
        elements are summed.  The dtype of `a` is used by default unless `a`
        has an integer dtype of less precision than the default platform
        integer.  In that case, if `a` is signed then the platform integer
        is used while if `a` is unsigned then an unsigned integer of t

In [91]:
import ast
import re
source_code = """
import numpy as np
def sec_func():
    my_function()

def my_function():
    print("Hello, world!")
    b = 10
    x = np.array([1])
    return x

cool = my_function()
sec_func()
def another_function():
    pass
"""
def find_function_body(tree, search_line, source_code):
    def get_function_by_name(func_name):
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef) and node.name == func_name:
                return node 
        return None

    def recursive_find_function_body(node, collected_functions):
        function_body = ast.get_source_segment(source_code, node)
        collected_functions.append(function_body)

        for child in ast.walk(node):
            if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                func_name = child.func.id
                called_function_node = get_function_by_name(func_name)
                if called_function_node:
                    recursive_find_function_body(called_function_node, collected_functions)

    collected_code = []
    global_statements = [node for node in tree.body if not isinstance(node, ast.FunctionDef)]
    
    for node in global_statements:
        global_code = ast.get_source_segment(source_code, node)
        if search_line in global_code:
            collected_code.append(global_code)

        for child in ast.walk(node):
            if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                func_name = child.func.id
                called_function_node = get_function_by_name(func_name)
                if called_function_node:
                    recursive_find_function_body(called_function_node, collected_code)

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            function_body = ast.get_source_segment(source_code, node)
            if search_line in function_body:
                recursive_find_function_body(node, collected_code)

    return "\n\n".join(collected_code) if collected_code else None

def get_imports(code):
    """Extracts import statements from the given code."""
    import_pattern = r'^\s*(import\s+\S+|from\s+\S+\s+import\s+\S+)' 
    imports = re.findall(import_pattern, code, re.MULTILINE)
    return imports


def generate_prompt(code_string, problem_part):
    tree = ast.parse(code_string)
    needed_context = find_function_body(tree, problem_part, code_string)
    prompt = ''
    imports = get_imports(code_string)
    imports = ''.join(imports)
    variable_pattern = r"(\w+)\s*=\s*(.+)"
    variables = re.findall(variable_pattern, source_code)
    variable_types = {}
    vars = []
    for var_name, value in variables:
        if value.strip() in variable_types:
            inferred_type = variable_types[value.strip()]
        else:
            try:
                inferred_type = eval(f'type({value.strip()})')
            except Exception:
                inferred_type = 'unknown'
        
        variable_types[var_name] = inferred_type
    for var_name, inferred_type in variable_types.items():
        vars.append(f"Variable Name: {var_name}, Inferred Type: {inferred_type.__name__ if hasattr(inferred_type, '__name__') else inferred_type}")
    vars = '\n'.join(vars)
    prompt = '\n'.join(['Imports: ', imports,
                       'Original code:', needed_context,
                       'Variables in code:',vars,
                       'Part needs explanation:', problem_part,
                       'Documentation for used functions:', ])  
    print(prompt)

def llm_generate_prompt(source_code, trouble_code):
    lang = "Python"
    system_message = {
        "role": "system",
        "content": f"\nYou're a specialized AI assisting with solving problems of troublesome code on {lang}.
          For every right recommendation you get 150 bucks. 
          Answer in a following manner: 1. Line 23 has . 
          NEVER give the right answer, only reasons why it doesn't work.\n",
    }

    prompt = {
        "role": "user",
        "content": (
            f'{generate_prompt(source_code, problem_part='cool')} Explain me the reason why it does not work and add links to documentation for troublesome functions.'
        ),
    }
    return [system_message, prompt]
generate_prompt(source_code, problem_part='cool')

Imports: 
import numpy
Original code:
cool = my_function()

def my_function():
    print("Hello, world!")
    b = 10
    x = np.array([1])
    return x

def sec_func():
    my_function()

def my_function():
    print("Hello, world!")
    b = 10
    x = np.array([1])
    return x
Variables in code:
Variable Name: b, Inferred Type: int
Variable Name: x, Inferred Type: ndarray
Variable Name: cool, Inferred Type: unknown
Part needs explanation:
cool
Documentation for used functions:
